The following code is to take a list of MP-IDs and get the gibbs energy of decomposition for each material. 

In [56]:
from mp_api.client import MPRester
from pymatgen.analysis.pourbaix_diagram import PourbaixDiagram, PourbaixPlotter, PourbaixEntry


API_KEY = "M3iWLItVErrMW5qgPwe5tzZFxjlROt1x"  

Take list of MP_IDs and get the material entries from the database

from entries, collect the elemental chemsys and composition and create lists with that data

create a new list with the material ids with "gga" added to search through the large list of data that will be generated in the next few cells

In [ ]:
materials = ["mp-2657","mp-20782","mp-866101","mp-861502","mp-568392","mp-560328","mp-861942","mp-690687","mp-558763","mp-1229122"]
mp_entries = []

with MPRester(API_KEY) as mpr:
    for mp_id in materials:
        material_entries = mpr.materials.search(material_ids=[mp_id])

        if material_entries:
            mp_entries.append(material_entries[0])

chemsys_list = [entry.chemsys for entry in mp_entries if hasattr(entry, 'chemsys')]
composition_list = [entry.composition_reduced for entry in mp_entries if hasattr(entry, 'composition_reduced')]
materials_gga = [material + "-GGA" for material in materials]

the elemental list must not contain O (oxegen) or H (hydrogen) to compute the pourbaix data, the remove_o_and_h function removes those from the chemical composisiotn list

In [105]:
from pymatgen.core.composition import Composition

def remove_o_and_h(composition):
    new_composition_elements = []
    for element, count in composition.get_el_amt_dict().items():
        if element not in ['O', 'H']:
            new_composition_elements.append(f"{element}{int(count)}")
    new_composition_str = ' '.join(new_composition_elements)
    return Composition(new_composition_str)

filtered_molecules = [remove_o_and_h(molecule) for molecule in composition_list]

the pourbaix charts also requires the elements to be in a percentage form, the calculate_percent_composition looks at the atoms per element in the composition list and converts it to a percentage

In [106]:
from pymatgen.core.composition import Composition

def calculate_percent_composition(molecule):
    total_atoms = sum(molecule.values())
    return {element.symbol: count / total_atoms for element, count in molecule.items()}

percent_compositions = [calculate_percent_composition(molecule) for molecule in filtered_molecules]

the following code takes the chemsys and percent composition and gets the pourbaix entries and converts it to data. some materials do not have a pourbaix entry to error is placed into the list, (lenght of list needs to match in a future function)

In [ ]:
pbx_data = []
pbx_entries = []

with MPRester(API_KEY) as mpr:
    for i, x in zip(chemsys_list, percent_compositions):
        try:
            pourbaix_entries = mpr.get_pourbaix_entries(i)
            pbx_entries.append(pourbaix_entries)
            pbx_diagram = PourbaixDiagram(entries=pourbaix_entries, comp_dict=x)
            pbx_data.append(pbx_diagram)
        except Exception as e:
            pbx_data.append('error')

the pourbaix entry responds with all viable pourbaix diagrams for a chemsys list, the following code searches through the list to get the correct diagram for the material of interest. 

In [108]:
matched_entries = []

for mat_id, entries in zip(materials_gga, pbx_entries):
    matched_entry = next((entry for entry in entries if entry.entry_id == mat_id), None)
    
    if matched_entry:
        matched_entries.append(matched_entry)
    else:
        matched_entries.append(None)

the stability data is then calculated at ph7 and 0V using the entry names and pbx_data

In [109]:
stabilities = []

for i, x in zip(pbx_data, matched_entries):
    try:
        stability = i.get_decomposition_energy(x, 7, 0)
        stabilities.append(stability)
    except Exception as e:
        stabilities.append("error") 


print(stabilities)


[0.037221306666667196, 0.9013717632604759, 'error', 'error', 0.02571852262929969, 1.2820778767603975, 1.233283079560989, 0.7292489545924903, 0.28738948977272744, 0.5716259503989086]
